<a href="https://colab.research.google.com/github/Jack0karev/konoha/blob/master/%D0%9A%D0%B8%D1%82%D0%B0%D0%B9%D1%81%D0%BA%D0%B8%D0%B5_%D1%8D%D1%80%D0%BE%D0%B3%D0%BB%D0%B8%D1%84%D1%8B_%D0%B4%D0%B7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload()

In [0]:
!cp kaggle.json  ~/.kaggle/

In [0]:
!kaggle competitions download -c chinese-char-recognition-smmo19
!ls

In [0]:
!unzip train-1.npy.zip  train-1.npy
!unzip train-2.npy.zip  train-2.npy
!unzip train-3.npy.zip  train-3.npy
!unzip train-4.npy.zip  train-4.npy

In [0]:
import numpy as np
import pandas as pd
train_1 = np.load('train-1.npy', allow_pickle=True)
train_2 = np.load('train-2.npy', allow_pickle=True)
train_3 = np.load('train-3.npy', allow_pickle=True)
train_4 = np.load('train-4.npy', allow_pickle=True)
train = np.concatenate((train_1, train_2, train_3, train_4), axis=0)
train, labels = train[:, 0], train[:, 1]

In [0]:
from skimage.transform import resize
for i in range(train.shape[0]):
  train[i] = resize(train[i], (32, 32), anti_aliasing=True)


In [0]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots(5,10)
for i in range(5):
    for j in range(10):
        ax[i][j].imshow(train[np.random.randint(0,train.shape[0])],cmap=plt.cm.binary)
        ax[i][j].axis('off')
plt.subplots_adjust(wspace=0, hspace=0)        
fig.set_figwidth(16)
fig.set_figheight(8)
fig.show()

In [0]:
y_train = labels.reshape(1, labels.shape[0])
y_train = pd.get_dummies(y_train[0]).to_numpy()
new = []
for i in train:
  new.append(i)
x_train = np.asarray(new)
x_train = x_train.reshape(train.shape[0],32, 32, 1)

In [0]:
columns_names = pd.get_dummies(labels).columns

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import Dropout, MaxPool2D
from keras.models import Model
from keras.layers.convolutional import MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
import keras


In [0]:
def make_model():    
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation = "relu", input_shape = (32,32,1), padding = "same"))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3,3), activation = "relu"))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (5,5), strides = (2,2) ,activation = "relu", padding = "same"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3,3), activation = "relu", input_shape = (32,32,1), padding = "same"))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3,3), activation = "relu"))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (5,5), strides = (2,2) ,activation = "relu", padding = "same"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2)) 
    model.add(Conv2D(128, (3,3), activation = "relu", padding = "same"))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), activation = "relu"))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (5,5), strides = (2,2), activation = "relu", padding = "same"))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))
    model.add(Conv2D(256, (3,3), activation = "relu", padding = "same"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Flatten())   
    model.add(Dense(1024, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(1000, activation='softmax')) 

    return model

In [0]:
INIT_LR = 5e-3
BATCH_SIZE = 64
EPOCHS = 50

model_filename = 'weights_{0:02d}.hdf5'

model = make_model()
model.summary()
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch 
  
history = model.fit(
    x_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split = 0.01,
    shuffle=True,
    verbose=1,
    initial_epoch=0    
)
  

In [0]:
del train
del train_1
del train_2
del train_3
del train_4
del new

In [0]:
!unzip test.npy.zip  test.npy

In [0]:
test = np.load('test.npy', allow_pickle=True)

for i in range(test.shape[0]):
    test[i] = resize(test[i], (32, 32), anti_aliasing=True)


new_test = []
for i in test:
  new_test.append(i)



In [0]:
x_test = np.asarray(new_test)
x_test = x_test.reshape(x_test.shape[0],32, 32, 1)

In [0]:
prediction = model.predict(x_test)
df = pd.DataFrame(prediction)
df.columns = columns_names
df1 = df
df1 = pd.get_dummies(df1).idxmax(1)
df2 = pd.read_csv('random_labels.csv')

In [0]:
result = df2[['Id']]
result['Category'] = df1
result.to_csv('final1.csv', index=False) 
files.download('final1.csv')